# `nepare` ASAP `polaris` Competition

This notebook demonstrates using Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library to compete in the ASAP discovery competition.

## Requirements
Python 3.10+ (originally run on 3.12)
 - polaris-lib
 - pandas
 - rdkit
 - lightning
 - torch
 - fastprop
 - chemprop 2.1
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [1]:
import polaris as po
import pandas as pd

In [2]:
from packaging.version import Version
assert Version(po.__version__) >= Version("0.11.6"), "test.as_dataframe does not work in earlier versions of Polaris, please upgrade"

In [3]:
%%capture
competition = po.load_competition("asap-discovery/antiviral-potency-2025")
# or
# competition = po.load_competition("asap-discovery/antiviral-admet-2025")

Choose the type of embedding to use - either a fixed descriptor-based embedding with mordred (great with limited data) or a learnable embedding with ChemProp (more general representation).
The choice as shown is configured based on previous experimentation.

In [4]:
_m, _c, "mordred", "chemprop"
EMBEDDING = {
    'HLM': _c,
    'MLM': _m,
    'KSOL': _m,
    'LogD': _m,
    'MDR1-MDCKII': _m,
    'pIC50 (MERS-CoV Mpro)': _m,
    'pIC50 (SARS-CoV-2 Mpro)': _c,
}

In [5]:
train, test = competition.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

In [6]:
train_df

,CXSMILES,pIC50 (SARS-CoV-2 Mpro),pIC50 (MERS-CoV Mpro)
0,COC[C@]1(C)C(=O)N(C2=CN=CC3=CC=CC=C23)C(=O)N1C...,NaN,4.19
1,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.29,4.92
2,CNC(=O)CN1C[C@]2(C[C@H](C)N(C3=CN=CC=C3C3CC3)C...,NaN,4.73
3,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,6.11,4.90
4,C=C(CN1CCC2=C(C=C(Cl)C=C2)C1C(=O)NC1=CN=CC2=CC...,5.62,4.81
...,...,...,...
1026,CNS(=O)(=O)OCC(=O)N1CCN(CC2=CC=CC(Cl)=C2)[C@H]...,6.38,5.57
1027,O=C(CC1=CN=CC2=CC=CC=C12)N1CC[C@@H]2CCCC[C@H]2...,6.09,4.60
1028,CNC(=O)[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)C1...,NaN,4.22
1029,C[C@H]1CCCN(C(=O)CC2=CN=CC3=CC=CC=C23)[C@H]1C ...,5.06,4.40


## Train Per-Task Models

In [7]:
from pathlib import Path

import torch
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger
from chemprop.nn.agg import MeanAggregation
from chemprop.nn.message_passing import BondMessagePassing
from fastprop.data import standard_scale, inverse_standard_scale

from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.nn import LearnedEmbeddingNeuralPairwiseRegressor, NeuralPairwiseRegressor
from nepare.inference import predict

from utils.splitting import split
from utils.chemprop_wrappers import smiles2molgraphcache, collate_fn, ChemPropEmbedder
from utils.mordred_wrappers import smi2features
from utils.metrics import evaluate_predictions

In [8]:
predictions = {}
output_dir = Path("lightning_logs")
tasks = list(competition.target_cols)
dataset_kwargs = dict(batch_size=64)
for task_n, task_name in enumerate(tasks):
    task_df = train_df[["CXSMILES", task_name]].copy()
    task_df.dropna(inplace=True)
    task_df.reset_index(inplace=True)
    train_idxs, val_idxs = split(task_df["CXSMILES"], test_df["CXSMILES"])
    train_targets = torch.tensor(task_df[task_name].iloc[train_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    train_targets, target_means, target_vars = standard_scale(train_targets)
    val_targets = torch.tensor(task_df[task_name].iloc[val_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    val_targets = standard_scale(val_targets, target_means, target_vars)
    if EMBEDDING[task_name] == "mordred":
        Model = NeuralPairwiseRegressor
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_features, train_targets, how='full')
        val_dataset = PairwiseAnchoredDataset(train_features, train_targets, val_features, val_targets, how='full')
        val_absolute_dataset = PairwiseInferenceDataset(train_features, train_targets, val_features, how='full')
        test_dataset = PairwiseInferenceDataset(train_features, train_targets, test_features, how='full')
        # build the model
        npr = Model(train_features.shape[1], 70, 3)
    elif EMBEDDING[task_name] == "chemprop":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        # setup the datasets
        dataset_kwargs["collate_fn"] = collate_fn
        train_dataset = PairwiseAugmentedDataset(train_mgc, train_targets, how='sut')
        val_dataset = PairwiseAnchoredDataset(train_mgc, train_targets, val_mgc, val_targets, how='half')
        val_absolute_dataset = PairwiseInferenceDataset(train_mgc, train_targets, val_mgc, how='half')
        test_dataset = PairwiseInferenceDataset(train_mgc, train_targets, test_mgc, how='half')
        # build the model
        mp = BondMessagePassing(d_h=400)
        agg = MeanAggregation()
        embedder = ChemPropEmbedder(mp, agg)
        npr = Model(embedder, 400, 200, 2, lr=5e-5)

    # classic lightning training, inference
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **dataset_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, **dataset_kwargs)
    val_absolute_loader = torch.utils.data.DataLoader(val_absolute_dataset, **dataset_kwargs)
    predict_loader = torch.utils.data.DataLoader(test_dataset, **dataset_kwargs)
    early_stopping = EarlyStopping(monitor="validation/loss", patience=3)
    name = "_".join([EMBEDDING, task_name])
    model_checkpoint = ModelCheckpoint(dirpath=output_dir / name, monitor="validation/loss")
    logger = TensorBoardLogger(save_dir=output_dir, name=name)
    trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint], logger=logger)
    trainer.fit(npr, train_loader, val_loader)
    npr = Model.load_from_checkpoint(model_checkpoint.best_model_path)
    y_pred, y_stdev = predict(npr, val_absolute_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    y_true = inverse_standard_scale(val_targets, target_means, target_vars)

    # metric reporting, for our own information
    print(f"{task_name=}:\n - {"\n - ".join([f'{name}: {score:.4f}' for name, score in evaluate_predictions(y_true.flatten().numpy(), y_pred.flatten().numpy()).items()])}")

    # actual predictions
    y_pred, y_stdev = predict(npr, predict_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    predictions[task_name] = y_pred.flatten().tolist()

/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:209: Attribute 'embedding_module' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['embedding_module'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jackson/neural-pairwise-regression/notebooks/polaris_asap/lightning_logs/chemprop_pIC50 (SARS-CoV-2 Mpro) exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | fnn              | Sequential       | 200 K  | train
1 | embedding_module | ChemPropEmbedder | 384 K  | train
-

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


task_name='pIC50 (MERS-CoV Mpro)':
 - Pearson r: 0.4273
 - MSE: 0.7876
 - MAE: 0.6101


Predicting: |          | 0/? [00:00<?, ?it/s]

This last block is commented out because it will fail (unless you are me) - you can replace the inputs with your own if you are submitting this for yourself.

In [9]:
# competition.submit_predictions(
#     predictions=predictions,
#     prediction_name="nepare",
#     prediction_owner="jacksonburns",
#     report_url="https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/meta",
#     github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_asap/main.ipynb",
#     description = "Neural Pairwise Regression with ChemProp as a learnable embedding",
# )